# 05 Preparación del Dataset y Entrenamiento con Yolov8

Este notebook describe el proceso fundamental para preparar el dataset y entrenar el modelo de detección de balsas utilizando la arquitectura YOLOv8-obb. Se detalla la creación de la estructura de directorios necesaria para el entrenamiento, validación y prueba, así como la configuración del archivo YAML que define el dataset para YOLOv8. Finalmente, se presenta un ejemplo de código para iniciar el entrenamiento del modelo utilizando un modelo pre-entrenado y ajustarlo a nuestro conjunto de datos específico, pero el entrenamiento, devido a las limitaciones del equipo local debemos realizarlo en Google Colab utilizando el siguiente Notebook.

Texto breve introductorio para presentar el modelo y describir als fases
* Creación del dataset
* Entrenar el modelo (Google Colab)
* Validar el modelo (Google Colab)

## Estructuración del Dataset para Entrenamiento de YOLOv8

Para entrenar un modelo de detección de objetos con YOLOv8, es fundamental estructurar el dataset de manera adecuada. Esto incluye organizar las imágenes y sus correspondientes etiquetas en carpetas específicas para los conjuntos de entrenamiento (`train`), validación (`val`) y prueba (`test`). Además, se deben seguir proporciones estándar para garantizar que el modelo se entrene correctamente y pueda generalizar bien a datos no vistos.

---

### Estructura del Dataset
El dataset debe estar organizado en una estructura jerárquica como la siguiente:

```
dataset/
├── images/
│   ├── train/       # Imágenes para entrenamiento
│   ├── val/         # Imágenes para validación
│   └── test/        # Imágenes para prueba
└── labels/
    ├── train/       # Etiquetas para entrenamiento
    ├── val/         # Etiquetas para validación
    └── test/        # Etiquetas para prueba
```

Cada imagen en las carpetas `images` debe tener un archivo de texto correspondiente en las carpetas `labels`, con el mismo nombre pero extensión `.txt`. Estos archivos contienen las anotaciones (etiquetas) en formato YOLO-obb, que describen las posiciones y clases de los objetos presentes en la imagen.

---

### Proporciones de Datos
Es común dividir el dataset en las siguientes proporciones:
- **70%** para entrenamiento (`train`): El conjunto de datos utilizado para que el modelo aprenda los patrones característicos de las balsas.
- **20%** para validación (`val`): Un conjunto de datos separado que se utiliza para monitorear el rendimiento del modelo durante el entrenamiento y ajustar hiperparámetros
- **10%** para prueba (`test`): Un conjunto de datos final e independiente que se utiliza para evaluar el rendimiento del modelo entrenado en datos completamente nuevos, proporcionando una estimación de su capacidad de generalización.

Esto asegura que haya suficientes datos para entrenar el modelo, mientras se reserva una parte para validar su rendimiento durante el entrenamiento y otra para evaluarlo al final.

---

El siguiente código implementa un flujo de trabajo eficiente para preparar un dataset compatible con YOLOv8. 

In [1]:
# Importar librerias necesarias
import random
import shutil
from pathlib import Path

In [2]:
# Definición de rutas
input_images_dir = Path("../data/interim/dataset/images")
input_labels_dir = Path("../data/interim/dataset/labels_obb")

output_base_dir = Path("../data/processed/dataset")
output_dirs = {
    "train": {
        "images": output_base_dir / "images" / "train",
        "labels": output_base_dir / "labels" / "train"
    },
    "val": {
        "images": output_base_dir / "images" / "val",
        "labels": output_base_dir / "labels" / "val"
    },
    "test": {
        "images": output_base_dir / "images" / "test",
        "labels": output_base_dir / "labels" / "test"
    }
}

# Crear directorios de salida si no existen
for split in output_dirs.values():
    for dir_path in split.values():
        dir_path.mkdir(parents=True, exist_ok=True)

# Obtener nombres de archivos de imágenes (sin extensión)
image_files = []
for file_path in input_images_dir.glob("*"):
    if file_path.is_file() and file_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.tif', '.tiff']:
        image_files.append(file_path.stem)

# Obtener nombres de archivos de etiquetas (sin extensión)
label_files = []
for file_path in input_labels_dir.glob("*.txt"):
    if file_path.is_file():
        label_files.append(file_path.stem)

# Verificar coincidencia de archivos
image_set = set(image_files)
label_set = set(label_files)

# Archivos en imágenes pero no en etiquetas
only_in_images = image_set - label_set
if only_in_images:
    print(f"ADVERTENCIA: {len(only_in_images)} archivos de imagen no tienen etiquetas correspondientes:")
    for file in list(only_in_images)[:5]:  # Mostrar solo los primeros 5 para no saturar la salida
        print(f"  - {file}")
    if len(only_in_images) > 5:
        print(f"  ... y {len(only_in_images) - 5} más")

# Archivos en etiquetas pero no en imágenes
only_in_labels = label_set - image_set
if only_in_labels:
    print(f"ADVERTENCIA: {len(only_in_labels)} archivos de etiquetas no tienen imágenes correspondientes:")
    for file in list(only_in_labels)[:5]:
        print(f"  - {file}")
    if len(only_in_labels) > 5:
        print(f"  ... y {len(only_in_labels) - 5} más")

# Usar solo archivos que tengan tanto imagen como etiqueta
valid_files = list(image_set.intersection(label_set))
print(f"Archivos válidos con imagen y etiqueta: {len(valid_files)}")

if not valid_files:
    print("ERROR: No hay archivos válidos para procesar.")
    exit(1)

# Mezclar aleatoriamente los archivos
random.seed(42) 
random.shuffle(valid_files)

# Calcular los tamaños de cada conjunto
total = len(valid_files)
train_size = int(total * 0.7)
val_size = int(total * 0.2)
# El resto va a test

# Dividir los archivos
train_files = valid_files[:train_size]
val_files = valid_files[train_size:train_size + val_size]
test_files = valid_files[train_size + val_size:]

print(f"Distribución de datos:")
print(f"  - Train: {len(train_files)} archivos ({len(train_files)/total:.1%})")
print(f"  - Validación: {len(val_files)} archivos ({len(val_files)/total:.1%})")
print(f"  - Test: {len(test_files)} archivos ({len(test_files)/total:.1%})")

# Función para encontrar la extensión correcta de un archivo de imagen
def find_image_extension(filename, directory):
    for ext in ['.jpg', '.jpeg', '.png', '.tif', '.tiff']:
        full_path = directory / f"{filename}{ext}"
        if full_path.exists():
            return ext
    return None

# Copiar los archivos a sus respectivos directorios
def copy_files(file_list, split_name):
    print(f"Copiando archivos para {split_name}...")
    count = 0
    for filename in file_list:
        # Buscar la extensión de la imagen
        img_ext = find_image_extension(filename, input_images_dir)
        if not img_ext:
            print(f"ADVERTENCIA: No se encontró la imagen para {filename}")
            continue
            
        # Ruta de origen y destino para imágenes
        src_img = input_images_dir / f"{filename}{img_ext}"
        dst_img = output_dirs[split_name]["images"] / f"{filename}{img_ext}"
        
        # Ruta de origen y destino para etiquetas
        src_lbl = input_labels_dir / f"{filename}.txt"
        dst_lbl = output_dirs[split_name]["labels"] / f"{filename}.txt"
        
        # Copiar archivos
        try:
            shutil.copy2(src_img, dst_img)
            shutil.copy2(src_lbl, dst_lbl)
            count += 1
        except Exception as e:
            print(f"Error al copiar {filename}: {e}")
    
    print(f"Se copiaron {count} archivos al conjunto {split_name}")

# Copiar todos los conjuntos
copy_files(train_files, "train")
copy_files(val_files, "val")
copy_files(test_files, "test")

print("\nProceso completado.")

Archivos válidos con imagen y etiqueta: 593
Distribución de datos:
  - Train: 415 archivos (70.0%)
  - Validación: 118 archivos (19.9%)
  - Test: 60 archivos (10.1%)
Copiando archivos para train...
Se copiaron 415 archivos al conjunto train
Copiando archivos para val...
Se copiaron 118 archivos al conjunto val
Copiando archivos para test...
Se copiaron 60 archivos al conjunto test

Proceso completado.


## Entrenar el modelo YOLOv8-obb 

Notebook: `06 Entrenamiento y Validación del Modelo (Google Colab).ipynb`

### Google Colab

Debido a la limitación de recursos de hardware disponibles localmente, específicamente la falta de una GPU con la capacidad de procesamiento necesaria, se ha recurrido al uso de las GPUs proporcionadas por Google Colab para el entrenamiento del modelo de detección de imágenes Yolov8n-obb. Esta plataforma en la nube ofrece la infraestructura computacional requerida para llevar a cabo el entrenamiento de manera eficiente, superando las limitaciones del equipo local.

### Yolov8n-obb
La elección del modelo Yolov8n (nano) para la detección de balsas en imágenes aéreas se fundamenta en su arquitectura ligera, diseñada para minimizar las necesidades de cómputo y acelerar significativamente los tiempos de entrenamiento. Esta optimización es crucial en el contexto de recursos limitados y la necesidad de obtener resultados en plazos razonables.

Adicionalmente, se ha optado por la variante OBB (Oriented Bounding Boxes) de Yolov8 debido a su capacidad superior para la detección precisa de objetos con orientaciones arbitrarias, característica inherente a las balsas visualizadas en las imágenes aéreas. Los modelos OBB son especialmente recomendados en este tipo de aplicaciones, ya que capturan la orientación de los objetos, evitando los recortes e imprecisiones que pueden surgir con los bounding boxes axis-aligned tradicionales.

Finalmente, para optimizar aún más el proceso de entrenamiento y reducir los tiempos requeridos, se empleará un modelo Yolov8 preentrenado. Este enfoque, conocido como transfer learning o fine-tuning, permite aprovechar el conocimiento previamente adquirido por el modelo en un conjunto de datos masivo. Al adaptar este modelo preentrenado utilizando nuestro conjunto de datos específico de imágenes de balsas aéreas, se logra una convergencia más rápida y se requiere una menor cantidad de datos y tiempo de entrenamiento en comparación con un entrenamiento desde cero.

### Fichero YALM

Los modelos de YOLOv8 (y en general muchos modelos de detección de **Ultralytics**) usan un **archivo `.yaml`** para **definir el conjunto de datos** que se va a utilizar para entrenamiento o inferencia.

Esto incluye:
- Las **rutas** a los datos de entrenamiento y validación
- La **lista de clases** (etiquetas) del conjunto de datos
- (Opcional) Formato de anotaciones, número de canales, entre otros

Esto permite que el mismo modelo se entrene con distintos datasets **sin tener que modificar el código**, simplemente cambiando la ruta del archivo `.yaml`.

### Ejemplo de archivo `data.yaml` para YOLOv8-obb:

```yaml
# Archivo: data.yaml

# Rutas de entrenamiento/validación/prueba (ajusta según tu estructura de directorios)
path: ../data/processed/dataset  # ruta base del dataset
train: images/train  # ruta relativa a las imágenes de entrenamiento
val: images/val  # ruta relativa a las imágenes de validación
test: images/test  # ruta relativa a las imágenes de prueba (opcional)

# Nombres de las clases
names:
  0: balsa

# Información adicional
nc: 1  # número de clases
task: detect  # tipo de tarea: detect, segment, classify, etc.
```

### Ejemplo básico de parámetros de entrenamiento para Yolov8n-obb
Un ejemplo de código para entrenar Yolov8n.obb sería el siguiente.


```python
# Importar librería y cargar el modelo pre-entrenado
from ultralytics import YOLO # Load a model 
model = YOLO("../models/yolov8n-obb.pt")


#Entrenar el modelo 
results = model.train( data="../models/data.yaml", epochs=100, patience=15, batch=2, imgsz=640, optimizer="AdamW", plots=True, save=True, project="deteccion_balsas", name="yolov8n_obb_run1", exist_ok=True)
```